### Download all dependencies

In [4]:
import pandas as pd # primary data structure library
import numpy as np  # useful for many scientific computing in Pyth
wiki = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Data_table = wiki[0]
Data_table.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Remove cells with a borough that is not assigned.

In [5]:

Data_table_temp=Data_table.drop(Data_table[Data_table["Borough"] == "Not assigned"].index)
Data_table_temp.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Aggreagte rows with the same postal code

In [6]:

Data_table_temp1 = Data_table_temp.groupby(["Postal code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Data_table_temp1.head()


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Check If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
Data_table_temp1.loc[Data_table_temp1['Neighborhood'] == 'Not assigned', 'Neighborhood'] = Data_table_temp1.loc[Data_table_temp1['Neighborhood'] == 'Not assigned', 'Borough']

print(Data_table_temp1.loc[Data_table_temp1['Neighborhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal code, Borough, Neighborhood]
Index: []


In [8]:

Data_table_temp1.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:

df = Data_table_temp1
print("Your table has {} rows.".format(df.shape[0]))

Your table has 103 rows.


In [10]:
# install required package
!pip install geocoder 

In [11]:
import geocoder # import geocoder

In [12]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:

coordinates.rename(columns={"Postal Code": "Postal code"}, inplace=True)
coordinates.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
Data_table_temp1_new = Data_table_temp1.merge(coordinates, on="Postal code", how="left")
Data_table_temp1_new.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
column_names = ["Postal code", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(Data_table_temp1_new[Data_table_temp1_new["Postal code"]==postcode], ignore_index=True)
    
test_df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


### Export the df_with_cord as a csv file for using in next assignment.

In [17]:
test_df.to_csv(r'Canada_postcode_geospatial_agg.csv', index=None, header=True)